Download `Highways` data (use [this link](https://new.portal.its.pdx.edu:8080/downloads/)) for each day and rename the data file as `year-month-day.csv` (e.g., 2021-01-01.csv).

In [1]:
import pandas as pd

data = pd.read_csv('2021-01-01.csv')
for i in range(2, 32):
    if i < 10:
        df = pd.read_csv('2021-01-0{}.csv'.format(i))
    else:
        df = pd.read_csv('2021-01-{}.csv'.format(i))
    data = data.append(df, ignore_index = True)
data = data.drop_duplicates().reset_index(drop = True)

In [2]:
data.head()

,starttime,resolution,detector_id,speed,volume,occupancy,countreadings,delay,traveltime,vht,vmt
0,2021-01-01T00:00:00-08:00,00:15:00,100636,59.64,39,1.07,45,0.00,0.28,0.18,10.73
1,2021-01-01T00:00:00-08:00,00:15:00,102260,75.45,11,0.16,45,-0.16,0.63,0.12,8.75
2,2021-01-01T00:00:00-08:00,00:15:00,100860,58.67,6,0.17,12,0.03,1.36,0.14,7.95
3,2021-01-01T00:00:00-08:00,00:15:00,100058,55.00,49,1.38,45,0.04,0.46,0.37,20.58
4,2021-01-01T00:00:00-08:00,00:15:00,102221,35.80,5,0.22,45,0.56,1.39,0.12,4.15


There are unique 1156 detectors in this data set. We care about:

- `speed`
- `volume`
- `occupancy`

of these detector at different time slots.

In [3]:
data = data.drop(['resolution', 'countreadings', 'delay', 'traveltime', 'vht', 'vmt'], axis = 1)

In [4]:
data.head()

,starttime,detector_id,speed,volume,occupancy
0,2021-01-01T00:00:00-08:00,100636,59.64,39,1.07
1,2021-01-01T00:00:00-08:00,102260,75.45,11,0.16
2,2021-01-01T00:00:00-08:00,100860,58.67,6,0.17
3,2021-01-01T00:00:00-08:00,100058,55.00,49,1.38
4,2021-01-01T00:00:00-08:00,102221,35.80,5,0.22


In [5]:
import numpy as np
import time

start = time.time()
num = data.shape[0]
time_slot = np.zeros(num)
for i in range(num):
    time_slot[i] = ((int(data.starttime[i][8 : 10]) - 1) * 96 
                    + int(data.starttime[i][11 : 13]) * 4 
                    + int(data.starttime[i][14 : 16]) / 15)
end = time.time()
print(end - start)
data["time_slot"] = time_slot
data.head()

251.15307807922363


,starttime,detector_id,speed,volume,occupancy,time_slot
0,2021-01-01T00:00:00-08:00,100636,59.64,39,1.07,0.0
1,2021-01-01T00:00:00-08:00,102260,75.45,11,0.16,0.0
2,2021-01-01T00:00:00-08:00,100860,58.67,6,0.17,0.0
3,2021-01-01T00:00:00-08:00,100058,55.00,49,1.38,0.0
4,2021-01-01T00:00:00-08:00,102221,35.80,5,0.22,0.0


In [6]:
len(data.detector_id.unique())

1156

In [ ]:
detector_id = data.detector_id.unique()
i = 0
speed = np.zeros((len(detector_id), int(data.time_slot.max()) + 1))
volume = np.zeros((len(detector_id), int(data.time_slot.max()) + 1))
occupancy = np.zeros((len(detector_id), int(data.time_slot.max()) + 1))
for k in detector_id:
    df = data[data.detector_id == k]
    for t in range(int(data.time_slot.max()) + 1):
        if len(df[df.time_slot == t].speed.values) > 0:
            speed[i, t] = df[df.time_slot == t].speed.values[0]
        if len(df[df.time_slot == t].volume.values) > 0:
            volume[i, t] = df[df.time_slot == t].volume.values[0]
        if len(df[df.time_slot == t].occupancy.values) > 0:
            occupancy[i, t] = df[df.time_slot == t].occupancy.values[0]
    i += 1

np.save('speed.npy', speed)
np.save('volume.npy', volume)
np.save('occupancy.npy', occupancy)